In [80]:
#1- adicione código para ler o arquivo nomes_aleatorios.txt através do comando spark.read.csv. Carregue-o para dentro de um 
#dataframe chamado df_nomes e, por fim, liste algumas linhas através do método show. Exemplo: df_nomes.show(5)

from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

spark = SparkSession.builder.master("local[*]").appName("Exercicio Intro").getOrCreate()

df_nomes = spark.read.csv('nomes_aleatorios.txt')
df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



In [81]:
#2- adicionar código para renomear a coluna para Nomes, imprimir o esquema e mostrar 10 linhas do dataframe

df_nomes = df_nomes.withColumnRenamed("_c0","Nomes")
df_nomes.printSchema()
df_nomes.show(10)

root
 |-- Nomes: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



In [82]:
#3- adicione nova coluna chamada Escolaridade e atribua para cada linha um dos três valores de forma aleatória: Fundamental, 
#Medio ou Superior

from pyspark.sql.functions import *

#divide 1 pela quantidade de possibilidades para poder usar as condições com os valores gerados por rand
probabilidade = 1/3

df_nomes = df_nomes.withColumn("Escolaridade", when(rand() <= probabilidade, "Fundamental").when(rand() >= probabilidade*2, "Médio").otherwise("Superior"))

In [83]:
#4- adicione nova coluna chamada Pais e atribua para cada linha o nome de um dos 13 países da América do Sul, de forma aleatória

#divide 1 pela quantidade de possibilidades para poder usar as condições com os valores gerados por rand
probabilidade = 1/13

df_nomes = df_nomes.withColumn("Pais", 
                               when(rand() <= probabilidade, "Brasil")
                              .when(rand() <= probabilidade*2, "Argentina")
                              .when(rand() <= probabilidade*3, "Colômbia")
                              .when(rand() <= probabilidade*4, "Peru")
                              .when(rand() <= probabilidade*5, "Chile")
                              .when(rand() <= probabilidade*6, "Equador")
                              .when(rand() <= probabilidade*7, "Venezuela")
                              .when(rand() <= probabilidade*8, "Bolívia")
                              .when(rand() <= probabilidade*9, "Guiana")
                              .when(rand() <= probabilidade*10, "Uruguai")
                              .when(rand() <= probabilidade*11, "Suriname")
                              .when(rand() <= probabilidade*12, "Paraguai")
                              .otherwise("Guiana Francesa"))

In [84]:
#5- adicione nova coluna chamada AnoNascimento e atribua para cada linha um valor de ano entre 1945 e 2010, de forma aleatória

df_nomes = df_nomes.withColumn("AnoNascimento", floor(rand() * (2010 - 1945) + 1945))                                                     

In [85]:
#6- Usando o método select do dataframe (df_nomes), selecione as pessoas que nasceram neste século. Armazene o resultado em 
#outro dataframe chamado df_select e mostre 10 nomes deste

df_select = df_nomes.select("*").filter(df_nomes["AnoNascimento"] >= 2000)
df_select.select("Nomes").show(10)

+-----------------+
|            Nomes|
+-----------------+
|    Jamie Russell|
|    Tracy Herring|
|     David Medina|
|  Helen Blackwell|
|      Jessie Jean|
|      Donald Vogt|
|      Hugo Dayton|
|Marguerite Rankin|
| Maurice Blizzard|
|George Fiorentino|
+-----------------+
only showing top 10 rows



In [86]:
#7- Usando Spark SQL repita o processo da Pergunta 6

df_nomes.createOrReplaceTempView ("pessoas")
spark.sql("SELECT Nomes FROM pessoas WHERE AnoNascimento >= 2000").show(10)

+-----------------+
|            Nomes|
+-----------------+
|    Jamie Russell|
|    Tracy Herring|
|     David Medina|
|  Helen Blackwell|
|      Jessie Jean|
|      Donald Vogt|
|      Hugo Dayton|
|Marguerite Rankin|
| Maurice Blizzard|
|George Fiorentino|
+-----------------+
only showing top 10 rows



In [87]:
#8- Usando o método select do Dataframe df_nomes, Conte o número de pessoas que são da geração Millennials (nascidos entre 1980
#e 1994) no Dataset

df_nomes.select('*').filter(df_nomes['AnoNascimento'].between(1980, 1994)).agg(count('*').alias('Millennials')).show()

+-----------+
|Millennials|
+-----------+
|    2307801|
+-----------+



In [88]:
#9- Repita o processo da Pergunta 8 utilizando Spark SQL

spark.sql("SELECT COUNT(AnoNascimento) AS Millennials FROM pessoas WHERE AnoNascimento BETWEEN 1980 AND 1994").show()

+-----------+
|Millennials|
+-----------+
|    2307801|
+-----------+



In [89]:
#10- Usando Spark SQL, obtenha a quantidade de pessoas de cada país para uma das gerações abaixo. Armazene o resultado em um 
#novo dataframe e depois mostre todas as linhas em ordem crescente de Pais, Geração e Quantidade

query = """
SELECT Pais, 
    CASE
        WHEN AnoNascimento BETWEEN 1944 AND 1964 THEN 'BabyBoomers'
        WHEN AnoNascimento BETWEEN 1965 AND 1979 THEN 'GeracaoX'
        WHEN AnoNascimento BETWEEN 1980 AND 1994 THEN 'GeracaoY/Millenials'
        WHEN AnoNascimento BETWEEN 1995 AND 2015 THEN 'GeracaoZ'
    END AS Geracao,
    COUNT(*) as Quantidade 
FROM pessoas 
GROUP BY Pais, Geracao
ORDER BY Pais, Geracao, Quantidade"""

df_geracao = spark.sql(query)
#13 países * 4 gerações, o df tem 52 linhas
df_geracao.select('*').orderBy('Pais', 'Geracao', 'Quantidade').show(52)

+---------------+-------------------+----------+
|           Pais|            Geracao|Quantidade|
+---------------+-------------------+----------+
|      Argentina|        BabyBoomers|    437240|
|      Argentina|           GeracaoX|    327662|
|      Argentina|GeracaoY/Millenials|    327202|
|      Argentina|           GeracaoZ|    326486|
|        Bolívia|        BabyBoomers|    120466|
|        Bolívia|           GeracaoX|     89965|
|        Bolívia|GeracaoY/Millenials|     90092|
|        Bolívia|           GeracaoZ|     90027|
|         Brasil|        BabyBoomers|    236288|
|         Brasil|           GeracaoX|    177766|
|         Brasil|GeracaoY/Millenials|    177066|
|         Brasil|           GeracaoZ|    177520|
|          Chile|        BabyBoomers|    492277|
|          Chile|           GeracaoX|    369342|
|          Chile|GeracaoY/Millenials|    369144|
|          Chile|           GeracaoZ|    369540|
|       Colômbia|        BabyBoomers|    555372|
|       Colômbia|   